In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
import numpy as np
#from pptx import Presentation
#from pptx import Presentation
df = pd.read_excel("/content/Coursera MASKED data (1) (1).xlsx")

In [ ]:
# Standardize column names
df.columns = df.columns.str.strip()
df.rename(columns={
    "Course Name": "Course_Name",
    "Duration(in hrs)": "Duration_hrs",
    "Learning Hours Spent": "Learning_Hours_Spent",
    "Course Grade": "Course_Grade",
    "Enrollment Time": "Enrollment_Time",
    "Completion Time": "Completion_Time",
    "Skills Learned": "Skills_Learned",
    "Program Name": "Program_Name"
}, inplace=True)

# Convert columns to appropriate types
df['Duration_hrs'] = pd.to_numeric(df['Duration_hrs'], errors='coerce')
df['Learning_Hours_Spent'] = pd.to_numeric(df['Learning_Hours_Spent'], errors='coerce')
df['Course_Grade'] = pd.to_numeric(df['Course_Grade'], errors='coerce')
df['Enrollment_Time'] = pd.to_datetime(df['Enrollment_Time'], errors='coerce')
df['Completion_Time'] = pd.to_datetime(df['Completion_Time'], errors='coerce')

# Clean and convert 'Completed' to numeric (1 for Yes, 0 for No)
df['Completed'] = df['Completed'].fillna('No').map({'Yes': 1, 'No': 0})

# Drop rows with too many missing values
df.dropna(thresh=5, inplace=True)

# Parse skills into list
df['Skills_List'] = df['Skills_Learned'].fillna('').apply(lambda x: [s.strip() for s in x.split(';') if s.strip()])

# Set style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# Save figures

In [ ]:

def save_plot(fig, name):
    fig.savefig(f"{name}.png", bbox_inches="tight")

# 1. Course Completion Status
fig1 = sns.countplot(x='Completed', data=df)
plt.title("Course Completion Status")
plt.xticks([0, 1], ['Not Completed', 'Completed'])
save_plot(fig1.figure, "01_completion_status")
plt.clf()

# 2. Top 10 Most Taken Courses
top_courses = df['Course_Name'].value_counts().nlargest(10)
fig2 = sns.barplot(y=top_courses.index, x=top_courses.values)
plt.title("Top 10 Most Taken Courses")
save_plot(fig2.figure, "02_top_courses")
plt.clf()

# 3. Program-wise Course Count
program_course_count = df['Program_Name'].value_counts().nlargest(10)
fig3 = sns.barplot(y=program_course_count.index, x=program_course_count.values)
plt.title("Top Programs by Course Count")
save_plot(fig3.figure, "03_program_course_count")
plt.clf()

# 4. Average Learning Hours per Program
avg_hours = df.groupby('Program_Name')['Learning_Hours_Spent'].mean().dropna().nlargest(10)
fig4 = sns.barplot(x=avg_hours.values, y=avg_hours.index)
plt.title("Average Learning Hours per Program")
save_plot(fig4.figure, "04_avg_hours_per_program")
plt.clf()

# 5. Learning Hours vs Course Grade
fig5 = sns.scatterplot(data=df, x='Learning_Hours_Spent', y='Course_Grade')
plt.title("Learning Hours vs Course Grade")
save_plot(fig5.figure, "05_hours_vs_grade")
plt.clf()

# 6. Course Grade Distribution
fig6 = sns.histplot(df['Course_Grade'].dropna(), bins=20, kde=True)
plt.title("Distribution of Course Grades")
save_plot(fig6.figure, "06_grade_distribution")
plt.clf()

# 7. Grade Distribution by Program
top_programs = df['Program_Name'].value_counts().nlargest(5).index
fig7 = sns.boxplot(data=df[df['Program_Name'].isin(top_programs)], x='Program_Name', y='Course_Grade')
plt.title("Grade Distribution Across Top Programs")
plt.xticks(rotation=45)
save_plot(fig7.figure, "07_grade_by_program")
plt.clf()

# 8. Skill Frequency (Top 20)
skills = df['Skills_List'].explode().dropna()
skill_counts = pd.Series(Counter(skills)).sort_values(ascending=False).head(20)
fig8 = sns.barplot(x=skill_counts.values, y=skill_counts.index)
plt.title("Top 20 Most Learned Skills")
save_plot(fig8.figure, "08_top_skills")
plt.clf()

# 9. Course Completion Over Time
df['Completion_Month'] = df['Completion_Time'].dt.to_period('M')
monthly_completion = df.groupby('Completion_Month').size()
fig9 = monthly_completion.plot(kind='line', marker='o')
plt.title("Monthly Course Completions")
plt.ylabel("Completions")
save_plot(fig9.figure, "09_monthly_completion")
plt.clf()

# 10. Learning Duration Distribution
fig10 = sns.histplot(df['Duration_hrs'].dropna(), bins=20, kde=True)
plt.title("Distribution of Course Duration")
save_plot(fig10.figure, "10_duration_distribution")
plt.clf()

# 11. Heatmap: Avg Grade by Division & Department
heatmap_df = df.pivot_table(index='Division', columns='Department', values='Course_Grade', aggfunc='mean')
fig11 = sns.heatmap(heatmap_df, cmap='viridis')
plt.title("Average Grade by Division and Department")
save_plot(fig11.figure, "11_heatmap_div_dept")
plt.clf()

# 12. Completion Rate by Division
division_completion = df.groupby('Division')['Completed'].mean().sort_values(ascending=False)
fig12 = sns.barplot(x=division_completion.values, y=division_completion.index)
plt.title("Completion Rate by Division")
save_plot(fig12.figure, "12_completion_by_division")
plt.clf()

# 13. Number of Courses Per User
user_course_count = df['Name'].value_counts().head(20)
fig13 = sns.barplot(x=user_course_count.values, y=user_course_count.index)
plt.title("Top 20 Users by Number of Courses Enrolled")
save_plot(fig13.figure, "13_user_course_count")
plt.clf()

# 14. Users with Highest Learning Hours
user_hours = df.groupby('Name')['Learning_Hours_Spent'].sum().dropna().sort_values(ascending=False).head(20)
fig14 = sns.barplot(x=user_hours.values, y=user_hours.index)
plt.title("Top 20 Users by Learning Hours")
save_plot(fig14.figure, "14_top_users_hours")
plt.clf()

# 15. Courses with Highest Avg Grade
top_grade_courses = df.groupby('Course_Name')['Course_Grade'].mean().dropna().sort_values(ascending=False).head(10)
fig15 = sns.barplot(x=top_grade_courses.values, y=top_grade_courses.index)
plt.title("Top 10 Courses by Average Grade")
save_plot(fig15.figure, "15_top_courses_grade")
plt.clf()

